In [103]:
import xml.etree.ElementTree as ET

import pandas as pd
import numpy as np

In [104]:
junits =["junit_e2e__20250321-020621.xml","junit_e2e__20250321-171237.xml", "junit_e2e__20250323-005317.xml"]

job = 0

all_job_test_results = {}

for junit in junits:
    flakes = []
    root = ET.parse(junit).getroot()

    tests_count = root.get("tests")
    if int(tests_count) <= 1000:
        continue

    for testcase in root.iter('testcase'):
        testname = testcase.get('name')

        if testname in flakes:
            continue

        skipped = testcase.find('skipped')
        failure = testcase.find('failure')

        if all_job_test_results.get(testname) is None:
            all_job_test_results[testname] = {}

        if skipped is not None:
            continue

        test_state = 1
        if failure is not None:
            if "flake" in failure.text:
                flakes.append(testname)
                continue
            test_state = 0

        if all_job_test_results[testname].get(job) is None:
            all_job_test_results[testname][job] = test_state
        else:
            if all_job_test_results[testname][job] != test_state:
                if all_job_test_results[testname][job] == 0:
                    continue
                else:
                    all_job_test_results[testname][job] = test_state

                #print(f"original state: {all_job_test_results[testname][job]}")
                #print(f"{testname} {test_state} {job}")

    job += 1


df = pd.DataFrame.from_dict(all_job_test_results, orient='index')

df.describe()


,0,1
count,1593.000000,1545.000000
mean,0.994350,0.997411
std,0.074976,0.050833
min,0.000000,0.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [105]:
df.head()

,0,1
[sig-cluster-lifecycle][Feature:Machines][Early] Managed cluster should have same number of Machines and Nodes [apigroup:machine.openshift.io] [Suite:openshift/conformance/parallel],1.0,NaN
[sig-cluster-lifecycle][Feature:Machines] Managed cluster should [sig-scheduling][Early] control plane machine set operator should not cause an early rollout [Suite:openshift/conformance/parallel],1.0,NaN
[sig-node] Managed cluster record the number of nodes at the beginning of the tests [Early] [Suite:openshift/conformance/parallel],1.0,1.0
[sig-ci] [Early] prow job name should match security mode [Suite:openshift/conformance/parallel],1.0,1.0
[sig-arch][Early] Managed cluster should [apigroup:config.openshift.io] start all core operators [Skipped:Disconnected] [Suite:openshift/conformance/parallel],0.0,1.0


In [106]:
df

,0,1
[sig-cluster-lifecycle][Feature:Machines][Early] Managed cluster should have same number of Machines and Nodes [apigroup:machine.openshift.io] [Suite:openshift/conformance/parallel],1.0,NaN
[sig-cluster-lifecycle][Feature:Machines] Managed cluster should [sig-scheduling][Early] control plane machine set operator should not cause an early rollout [Suite:openshift/conformance/parallel],1.0,NaN
[sig-node] Managed cluster record the number of nodes at the beginning of the tests [Early] [Suite:openshift/conformance/parallel],1.0,1.0
[sig-ci] [Early] prow job name should match security mode [Suite:openshift/conformance/parallel],1.0,1.0
[sig-arch][Early] Managed cluster should [apigroup:config.openshift.io] start all core operators [Skipped:Disconnected] [Suite:openshift/conformance/parallel],0.0,1.0
...,...,...
Cluster should be stable before test is started,1.0,1.0
[sig-network-edge][Conformance][Area:Networking][Feature:Router] The HAProxy router should be able to connect to a service that is idled because a GET on the route will unidle it [Skipped:Disconnected] [Suite:openshift/conformance/parallel/minimal],NaN,1.0
[sig-instrumentation][Late] Alerts shouldn't exceed the series limit of total series sent via telemetry from each cluster [Suite:openshift/conformance/parallel],NaN,1.0
[sig-network] LoadBalancers [Feature:LoadBalancer] should be able to preserve UDP traffic when server pod cycles for a LoadBalancer service on different nodes [Skipped:alibabacloud] [Skipped:aws] [Skipped:baremetal] [Skipped:ibmcloud] [Skipped:kubevirt] [Skipped:nutanix] [Skipped:openstack] [Skipped:ovirt] [Skipped:vsphere] [Suite:openshift/conformance/parallel] [Suite:k8s],NaN,0.0
